<a href="https://colab.research.google.com/github/cheffjiu/pytorch-tutorials-zh/blob/main/autogradqs_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/intro.ipynb) \|\|
[Quickstart](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/quickstart_tutorial.ipynb) \|\|
[Tensors](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/tensorqs_tutorial.ipynb) \|\| [Datasets &
DataLoaders](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/data_tutorial.ipynb) \|\|
[Transforms](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/transforms_tutorial.ipynb) \|\| [Build
Model](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/buildmodel_tutorial.ipynb) \|\| **Autograd** \|\|
[Optimization](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/optimization_tutorial.ipynb) \|\| [Save & Load
Model](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/saveloadrun_tutorial.ipynb)

Automatic Differentiation with `torch.autograd`（使用`torch.autograd`进行自动求导）
===============================================

在训练神经网络时，最常用的算法是**反向传播（back progapation）**。在该算法中，参数（模型权重）会根据损失函数相对于给定参数的**梯度(gradients)**进行调整。

为了计算这些梯度，PyTorch 有一个内置的求导引擎，名为`torch.autograd`。 它支持对任意计算图自动计算梯度。   

考虑最简单的单层神经网络，其输入为`x`，参数为`w`和`b`，并使用某个损失函数。可以按以下方式在PyTorch中定义它：


In [ ]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

Tensors, Functions and Computational graph(张量、函数与计算图)
==========================================

这段代码定义了以下内容 **计算图（computational graph）**:

![](https://pytorch.org/tutorials/_static/img/basics/comp-graph.png)

在这个网络中，`w` 和 `b` 是**参数**，我们需要对其进行优化。因此，我们需要能够计算损失函数关于这些变量的梯度。为了做到这一点，我们设置这些张量的 `requires_grad` 属性。

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>你可以在创建张量时设置<code>requires_grad</code>的值，也可以在之后使用 <code>x.requires_grad_(True)</code> 方法</p>

</div>



我们应用于张量以构建计算图的函数实际上是 `Function` 类的一个对象。这个对象知道如何在 *正向* 传播中计算该函数，也知道如何在 *反向传播* 步骤中计算其导数。反向传播函数的引用存储在张量的 `grad_fn` 属性中。你可以在 [文档](https://pytorch.org/docs/stable/autograd.html#function) 中找到关于 `Function` 的更多信息。


In [ ]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Computing Gradients
===================

为了优化神经网络中参数的权重，我们需要计算损失函数关于参数的导数，也就是说，在 `x` 和 `y` 的某些固定值下，我们需要 $\frac{\partial loss}{\partial w}$ 和 $\frac{\partial loss}{\partial b}$ 。为了计算这些导数，我们调用 `loss.backward()` ，然后从 `w.grad` 和 `b.grad` 中获取值：


In [ ]:
loss.backward()
print(w.grad)
print(b.grad)

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<ul>
<li>我们只能获取计算图中叶子节点的<code>grad</code>属性，这些叶子节点的<code>requires_grad</code>属性设置为<code>True</code>。对于图中的所有其他节点，梯度将不可用。 出于性能原因，我们只能在给定的图上使用<code>backward</code>进行一次梯度计算。如果我们需要在同一图上进行多次<code>backward</code>调用，则需要在<code>backward</code>调用中传入<code>retain_graph=True</code>。 </li>
</ul>
```

</div>



Disabling Gradient Tracking（禁用梯度跟踪）
===========================

默认情况下，所有`requires_grad=True`的张量都会跟踪其计算历史并支持梯度计算。然而，在某些情况下我们不需要这样做，例如，当我们已经训练好模型，只想将其应用于一些输入数据时，也就是说，我们只想通过网络进行*前向*计算。我们可以通过将计算代码放在`torch.no_grad()`块中来停止跟踪计算：


In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

实现相同结果的另一种方法是对张量使用 `detach()` 方法：


In [ ]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

有几个原因可能会让你想要禁用梯度跟踪：
 - 将神经网络中的某些参数标记为**冻结参数**。
 - 当你只进行前向传播时，**加快计算速度**，因为对不跟踪梯度的张量进行计算会更高效。

More on Computational Graphs（关于计算图的更多内容）
============================

从概念上讲，自动求导会在一个由[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)对象组成的有向无环图（DAG）中记录数据（张量）和所有执行的操作（以及生成的新张量）。在这个有向无环图中，叶节点是输入张量，根节点是输出张量。通过从根节点到叶节点追踪此图，你可以使用链式法则自动计算梯度。

在正向传播中，自动求导会同时做两件事：

- 运行所请求的操作以计算结果张量
- 在有向无环图（DAG）中维护操作的*梯度函数*。  

当在有向无环图（DAG）的根节点上调用`.backward()`时，反向传播就会启动。然后，`autograd`会：
- 从每个`.grad_fn`计算梯度，
- 将梯度累加到相应张量的`.grad`属性中，
- 使用链式法则，将梯度一直传播到叶张量。

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>需要注意的重要一点是，计算图会从头重新创建；每次调用<code>.backward()</code> 后，自动求导会开始构建一个新的计算图。这正是你能够在模型中使用控制流语句的原因；如果需要，你可以在每次迭代时改变形状、大小和操作。 </p>

</div>



Optional Reading: Tensor Gradients and Jacobian Products（可选阅读：张量梯度与雅可比积）
========================================================

在许多情况下，我们有一个标量损失函数，并且需要计算相对于某些参数的梯度。同时，也存在输出函数是任意张量的情况。在这种情况下，PyTorch允许你计算所谓的**雅可比积（Jacobian Products）**，而不是实际的梯度。

对于向量函数$\vec{y}=f(\vec{x})$，其中
$\vec{x}=\langle x_1,\dots,x_n\rangle$ 且
$\vec{y}=\langle y_1,\dots,y_m\rangle$，$\vec{y}$ 相对于 $\vec{x}$ 的梯度由 **雅可比矩阵（Jacobian matirx）** 给出：

$$\begin{aligned}
J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)
\end{aligned}$$

PyTorch并不计算雅可比矩阵本身，而是允许你针对给定的输入向量$v=(v_1 \dots v_m)$计算**雅可比积(Jacobian Product)**$v^T\cdot J$ 。这可以通过将$v$作为参数调用`backward`来实现。$v$的大小应与我们要计算积的原始张量的大小相同：


In [ ]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

请注意，当我们使用相同的参数第二次调用`backward`时，梯度值是不同的。这是因为在进行反向传播时，PyTorch会**累加梯度(accumulate gradients)**，也就是说，计算出的梯度值会加到计算图所有叶节点的`grad`属性上。如果你想计算正确的梯度，就需要先将`grad`属性清零。在实际训练中，*优化器*会帮助我们完成这项工作。


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>此前我们调用<code>backward()</code>函数时不带参数。这本质上等同于调用<code>backward(torch.tensor(1.0))</code>，对于标量值函数（例如神经网络训练过程中的损失函数），这是一种计算梯度的有效方法。 </p>

</div>



------------------------------------------------------------------------


Further Reading
===============

-   [Autograd
    Mechanics](https://pytorch.org/docs/stable/notes/autograd.html)
